# Load Dataset

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
device = 'mps' if torch.cuda.is_available() else 'cpu'

# downloading mnist
mnist_trainset = 
mnist_testset = 

## printing lengths

print('length of the training set: {}'.format(len(mnist_trainset)))
print('length of the test set: {}'.format(len(mnist_testset)))

## rendering a few example from each label


# Preprocessing Step

In [ ]:
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder

partition_index = 

def one_hot(y):
  #For converting a numpy array of 0-9 into a one hot encoding of vectors of length 10
  b = np.zeros((y.size, y.max() + 1))
  b[np.arange(y.size), y] = 1
  return b

print('processing labeld training x and y')
train_x = np.asarray([np.asarray() for i in tqdm(range())])
train_y = one_hot(np.asarray([np.asarray() for i in tqdm(range())]))

print('processing unlabled training data')
train_unlabled = np.asarray([np.asarray() for i in tqdm(range(,len()))])

print('processing labeld test x and y')
test_x = np.asarray([np.asarray() for i in tqdm(range(len()))])
test_y = one_hot(np.asarray([np.asarray() for i in tqdm(range(len()))]))

print('reformatting shape...')
train_x = np.expand_dims(train_x, 1)
train_unlabled = np.expand_dims(train_unlabled, 1)
test_x = np.expand_dims(test_x, 1)

#converting data to pytorch type
torch_train_x = torch.tensor(, requires_grad=True).to(device)
torch_train_y = torch.tensor().to(device)
torch_test_x = torch.tensor(test_x.astype(np.float32), requires_grad=True).to(device)
torch_test_y = torch.tensor(test_y).to(device)
torch_train_unlabled = 

# Defining Model

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()
        self.conv1 = nn.Conv2d()
        self.conv2 = nn.Conv2d()
        self.conv3 = nn.Conv2d()

        if torch.cuda.is_available():
            self.cuda()

    def forward(self, x):
        x = F.max_pool2d(F.relu(), 2)
        x = F.max_pool2d(F.relu(), 2)
        x = F.max_pool2d(F.relu(), 2)
        x = torch.flatten(, 1)
        return x

#defining model head
class Head(nn.Module):
    def __init__(self, n_class=10):
        super(Head, self).__init__()
        self.fc1 = nn.Linear(, )
        self.fc2 = nn.Linear(, )
        self.fc3 = nn.Linear(, )

        if torch.cuda.is_available():
            self.cuda()

    def forward(self, x):
        x = F.relu()
        x = F.relu()
        x = 
        return 


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = 
        self.head = 

        if torch.cuda.is_available():
            self.cuda()

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

model_baseline = 
print(model_baseline(torch_train_x[:1]).shape)
model_baseline

# Train without unlabeled data

In [ ]:
def supervised_train(model):

    batch_size = 
    lr = 
    momentum = # set to 0.9 or 0.95
    num_epochs = # a value more than 2000

    #defining a stocastic gradient descent optimizer
    optimizer = torch.optim.SGD()

    #defining loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    train_hist = []
    test_hist = []
    test_accuracy = []

    for epoch in tqdm(range(num_epochs)):

        #iterating over all batches
        for i in range(int(len(train_x)/batch_size)-1):

            #Put the model in training mode, so that things like dropout work
            

            # Zero gradients for the optimizer 
            

            #extracting X and y values from the batch
            X = torch_train_x[i*batch_size: (i+1)*batch_size]
            y = torch_train_y[i*batch_size: (i+1)*batch_size]

            # Make predictions for this batch
            y_pred = 

            #compute gradients with the loss function
            

            # Adjust learning weights
            optimizer.step()

        with torch.no_grad():

            #Disable things like dropout, if they exist
            model.train(False)

            #calculating epoch training and test loss
            train_loss = loss_fn(model(torch_train_x), torch_train_y).cpu().numpy()
            y_pred_test = model(torch_test_x)
            test_loss = loss_fn(y_pred_test, torch_test_y).cpu().numpy()

            train_hist.append() # use train loss to plot
            test_hist.append() # use test loss to plot

            #computing test accuracy
            matches = np.equal(np.argmax(y_pred_test.cpu().numpy(), axis=1), np.argmax(torch_test_y.cpu().numpy(), axis=1))
            test_accuracy.append()

    import matplotlib.pyplot as plt
    plt.plot(train_hist, label = 'train loss')
    plt.plot(test_hist, label = 'test loss')
    plt.legend()
    plt.show()
    plt.plot(test_accuracy, label = 'test accuracy')
    plt.legend()
    plt.show()

    maxacc = # maximum value of accuracy
    print('max accuracy: {}'.format(maxacc))
    
    return maxacc

supervised_maxacc = 

# Data Augmentation

In [ ]:
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt

class Augment:

   def __init__(self):

       blur = T.GaussianBlur()

       self.train_transform = torch.nn.Sequential(
           T.RandomAffine(),
           T.RandomPerspective(),
           T.RandomPerspective(),
           T.RandomPerspective(0.2,0.5),
           T.RandomApply([blur],),
           T.RandomApply([blur],)
       )

   def __call__(self, x):
       return self.train_transform(x), self.train_transform(x)

"""
Generating Test Augmentation
"""
a = Augment()
aug = a(torch_train_unlabled[0:100])

i=1
f, axarr = plt.subplots(2,2)
#positive pair
axarr[0,0].imshow(aug[0].cpu().detach().numpy()[i,0])
axarr[0,1].imshow(aug[1].cpu().detach().numpy()[i,0])
#another positive pair
axarr[1,0].imshow(aug[0].cpu().detach().numpy()[i+1,0])
axarr[1,1].imshow(aug[1].cpu().detach().numpy()[i+1,0])
plt.show()

# Defining Contrastive Loss

In [ ]:
class ContrastiveLoss(nn.Module):
   def __init__(self, batch_size, temperature=0.5):

       super().__init__()
       self.batch_size = batch_size
       self.temperature = temperature
       self.mask = (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float().to(device)

   def calc_similarity_batch(self, a, b):
       representations = torch.cat([a, b], dim=0)
       return F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)

   def forward(self, proj_1, proj_2):
       batch_size = proj_1.shape[0]
       z_i = F.normalize(proj_1, p=2, dim=1)
       z_j = F.normalize(proj_2, p=2, dim=1)

       similarity_matrix = self.calc_similarity_batch(z_i, z_j)

       sim_ij = torch.diag(similarity_matrix, batch_size)
       sim_ji = torch.diag(similarity_matrix, -batch_size)

       positives = torch.cat([sim_ij, sim_ji], dim=0)

       nominator = torch.exp(positives / self.temperature)

       denominator = self.mask * torch.exp(similarity_matrix / self.temperature)

       all_losses = -torch.log(nominator / torch.sum(denominator, dim=1))
       loss = torch.sum(all_losses) / (2 * self.batch_size)
       return loss

loss = # use the above loss
fake_proj_0, fake_proj_1 = a(torch_train_x)
fake_proj_0 = fake_proj_0[:,0,:,0]
fake_proj_1 = fake_proj_1[:,0,:,0]
loss(fake_proj_0, fake_proj_1)

# Training on whole data

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

model = 
model.train()

#defining key hyperparameters
batch_size = 512
epoch_size = 
num_epochs = # more than 50
patience = 5
cutoff_ratio = 0.001

#defining key learning functions
optimizer = torch.optim.Adam(, lr=1e-3)
num_examples = 
lossfn = 
augmentfn = 

#for book keeping
loss_hist = []
improvement_hist = []
schedule_hist = []

#for exponentially decreasing learning rate
scheduler = ExponentialLR(, gamma = 0.95)

#for early stopping
patience_count = 0

#Training Loop
avg_loss = 1e10
for i in range(num_epochs):

    print('epoch {}/{}'.format(i,num_epochs))

    total_loss = 0
    loss_change = 0

    for j in tqdm(range(epoch_size)):

        #getting random batch
        X = torch_train_unlabled[j*batch_size: (j+1)*batch_size]

        #creating pairs of augmented batches
        X_aug_i, X_aug_j = 

        #ensuring gradients are zero
        optimizer.zero_grad()

        #passing through the model
        z_i = model()
        z_j = model()

        #calculating loss on the model embeddings, and computing gradients
        loss = 
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        if True:
            z_i = model(X_aug_i)
            z_j = model(X_aug_j)

            #calculating new loss value
            new_loss = lossfn(z_i, z_j)

            loss_change += new_loss.cpu().detach().numpy() - loss.cpu().detach().numpy()

        total_loss += 

        #step learning rate scheduler
        schedule_hist.append(scheduler.get_last_lr())

    #########################
    # update scheduler here #
    #########################
    
    #calculating percentage loss reduction
    new_avg_loss = total_loss/epoch_size
    per_loss_reduction = (avg_loss-new_avg_loss)/avg_loss
    print('Percentage Loss Reduction: {}'.format(per_loss_reduction))

    #deciding to stop if loss is not decreasing fast enough
    if per_loss_reduction < cutoff_ratio:
        patience_count+=1
        print('patience counter: {}'.format(patience_count))
        if patience_count > patience:
            break
    else:
        patience_count = 0

    #setting new loss as previous loss
    avg_loss = new_avg_loss

    #book keeping
    avg_improvement = loss_change/epoch_size
    loss_hist.append(avg_loss)
    improvement_hist.append(avg_improvement)
    print('Average Loss: {}'.format(avg_loss))
    print('Average Loss change (if calculated): {}'.format(avg_improvement))

In [ ]:
plt.plot(schedule_hist, label='learning rate')
plt.legend()
plt.show()
plt.plot(loss_hist, label = 'loss')
plt.legend()
plt.show()